# Check input files
**Author: Jun Sasaki  Coded 2024-12-26  Updated 2025-04-08**<br>
Checking input files interactively. When completed, rename or create check_input.ipynb. This should cover netcdf and text files.

In [ ]:
import xarray as xr
import os
from xfvcom import FvcomDataLoader, FvcomAnalyzer, FvcomPlotConfig, FvcomPlotter
from xfvcom.helpers import get_index_by_value
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

In [ ]:
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfile = "obc_dye2020.nc"
ncfile_path = os.path.join(base_path, ncfile)
obc_dye2020 = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
obc_dye2020.ds

### Open boundary node numbers

In [ ]:
obc_nodes = obc_dye2020.ds.obc_nodes.values
print(f"obc_nodes={obc_nodes}")

In [ ]:
obc_node = 3150
obc_dye = obc_dye2020.ds['obc_dye']
obc_h = obc_dye2020.ds['obc_h']
obc_node_index = get_index_by_value(obc_nodes, obc_node)
print(f"obc_node_index={obc_node_index}")
#time = obc_dye2020.ds['time']
#siglay = obc_dye2020.ds['siglay'][:, obc_node_index]
# dye_concentration = obc_dye[:, :, obc_node_index]
# time_grid, siglay_grid = np.meshgrid(time, siglay)

In [ ]:
da = obc_dye2020.ds['obc_dye'].isel(nobc=obc_node_index)
da

In [ ]:
pngfile = "dye_obc.png"
cfg = FvcomPlotConfig(figsize=(10,2), dpi=96)
plotter = FvcomPlotter(obc_dye2020.ds, cfg)
xlim = (da.time[0].values, da.time[-1].values)
ylim = (1, 0)
clim = (da.min().item(), da.max().item())
clim = (3,7)
#clim = (5,6)
title = f"Dye at node={obc_node}"
contourf_kwargs=dict(levels=30, vmin=3, vmax=7)
colorbar_kwargs=dict(label="Dye")
fig, _, _ = plotter.ts_contourf(da, ylim=ylim, title=title, contourf_kwargs=contourf_kwargs, colorbar_kwargs=colorbar_kwargs)
fig.savefig(pngfile, dpi=600, bbox_inches='tight')

In [ ]:
%%skip
plt.figure(figsize=(10, 3))
plt.contourf(time_grid, siglay_grid, dye_concentration.T, cmap='viridis', levels=20)
plt.colorbar(label="Dye Concentration")
plt.xlabel("Time")
plt.ylabel("Sigma Layer Depth (siglay)")
plt.title(f"Dye Concentration at Open Boundary Node {obc_node}")
plt.xticks(rotation=45)
plt.tight_layout()
pngfile = f"dye_concentration_at_open_bound_node_{obc_node}.png"
plt.savefig(pngfile, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
selected_time = '2020-01-01'
selected_time = np.datetime64(selected_time)
dye_profile = obc_dye.sel(time=selected_time, method='nearest')
siglay = obc_dye2020.ds['siglay'][:, obc_node_index]
depth = siglay * obc_h  # 各ノードごとの水深を計算
depth = depth.transpose('siglay', 'nobc') 
nobc_indices = np.tile(obc_h.nobc.values, (len(siglay), 1)) 
nobc_numbers = np.tile(obc_nodes, (len(siglay),1))
siglay_grid = np.tile(siglay.values[:, None], (1, len(obc_nodes)))

In [ ]:
plt.close()
yaxis_siglay=False
plt.figure(figsize=(10, 3))
if yaxis_siglay:
    plt.contourf(
        nobc_indices, siglay_grid, dye_profile.values, levels=20, cmap='viridis'
    )
else:
    plt.contourf(
        nobc_indices, depth, dye_profile.values, levels=20, cmap='viridis'
    )
plt.colorbar(label="Dye Concentration")
plt.xlabel("Boundary Node Index")
plt.ylabel("Depth (m)")
plt.title(f"Dye Concentration Cross-Section on {selected_time}")
plt.gca().invert_yaxis()  # 水深を正の方向に（深くなるほど下へ）
#plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import holoviews as hv
import panel as pn
import hvplot.xarray
from datetime import datetime
hv.extension('bokeh')
pn.extension()

In [ ]:
ds = obc_dye2020.ds.copy()

In [ ]:
node_select = pn.widgets.Select(
    name='Node Index',
    options=list(range(ds.sizes['nobc'])),  # 0〜12
    value=0
)

def plot_time_siglay(node):
    """
    指定したノードの染料濃度を時系列とsiglayに基づいてヒートマップとしてプロットします。
    """
    # 指定したノードのデータを抽出（nobcを削除）
    dye_profile = ds['obc_dye'].isel(nobc=node, drop=True)  # (time, siglay)
    
    # プロットは 'time' vs 'siglay'
    heatmap = dye_profile.hvplot.quadmesh(
        x='time',
        y='siglay',
        cmap='viridis',
        colorbar=True,
        title=f"Dye Concentration at Node {node} Over Time",
        xlabel='Time',
        ylabel='Sigma Layer'
    )
    
    return heatmap

# 動的なプロットを作成
#interactive_plot1 = pn.bind(plot_time_siglay, node=node_select)

# レイアウトの作成
#layout1 = pn.Column(
#    "# Dye Concentration Over Time",
#    node_select,
#    interactive_plot1
#)

In [ ]:
interactive_plot1 = pn.bind(plot_time_siglay, node=node_select)

# レイアウトの作成
layout1 = pn.Column(
    "### Dye Concentration Over Time",
    node_select,
    interactive_plot1
)

In [ ]:
final_layout = pn.Row(layout1)

In [ ]:
final_layout

In [ ]:
# 時間選択用のウィジェット
time_options = [pd.to_datetime(t).strftime('%Y-%m-%d %H:%M') for t in ds['time'].values]

time_select = pn.widgets.DiscreteSlider(
    name='Time',
    options=time_options,  # 文字列形式のリスト
    value=time_options[0]
)

def plot_node_depth(time):
    """
    指定した時間の染料濃度をノードインデックスと水深に基づいてヒートマップとしてプロットします。
    """
    # 選択された時間を np.datetime64 に変換
    time_np = np.datetime64(time)
    
    # 指定した時間のデータを抽出
    dye_profile = ds['obc_dye'].sel(time=time_np)  # (siglay, nobc)
    
    # ノードインデックスを0〜12に設定
    node_indices = np.arange(ds.sizes['nobc'])
    dye_profile = dye_profile.assign_coords(node_index=('nobc', node_indices))
    
    # siglayとobc_hを取得
    # siglayは (siglay, nobc) だが、全ノードで同じと仮定
    siglay_1d = ds['siglay'].isel(nobc=0).values  # (30,)
    obc_h_values = ds['obc_h'].values  # (13,)
    
    # 水深を計算: depth = siglay * obc_h
    depth = siglay_1d[:, np.newaxis] * obc_h_values  # shape (30,13)
    
    # depthを座標として追加
    dye_profile = dye_profile.assign_coords(depth=(['siglay', 'nobc'], depth))

    max_depth = depth.max()
    min_depth = depth.min()
    
    # プロット
    heatmap = dye_profile.hvplot.quadmesh(
        x='node_index',
        y='depth',
        cmap='viridis',
        colorbar=True,
        title=f"Dye Concentration at Time {time} by Depth",
        xlabel='Node Index',
        ylabel='Depth (m)',
        ylim=(max_depth, min_depth)
    )

    return heatmap

# 動的なプロットを作成
interactive_plot2 = pn.bind(plot_node_depth, time=time_select)

# レイアウトの作成
layout2 = pn.Column(
    "# Dye Concentration by Depth",
    time_select,
    interactive_plot2
)


In [ ]:
final_layout = pn.Row(layout2)
final_layout

# Slice netcdf with time
To check netcdf files on PC, it is necessary to make their sizes much smaller; so slicing with time

In [ ]:
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfile = "TokyoBay18_2020_wnd.nc"
ncfile_path = f"{base_path}/{ncfile}"
fvcom_wnd = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)

In [ ]:
start = "2020-01-01 00:00:00"
end = "2020-01-07 00:00:00"
output_path = f"sliced_{ncfile}"
fvcom_wnd.slice_by_time(start, end).to_netcdf(output_path)

In [ ]:
fvcom_wnd.ds

In [ ]:
start = "2020-01-01 00:00:00"
end = "2020-01-07 00:00:00"
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay2020final_tsobc.nc", "TokyoBay2020julian_obc.nc",
           "TokyoBay2020kisarazufinal_sewer.nc", "TokyoBay2020final_river.nc",
           "TokyoBay2020final_sewer.nc"]
for ncfile in ncfiles:
    output_path = f"sliced_{ncfile}"
    ncfile_path = f"{base_path}/{ncfile}"
    fvcom = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
    fvcom.slice_by_time(start, end).to_netcdf(output_path)